### Leia o arquivo dos dados da Receita Federal e cruze eles com os dados da CEAP

In [2]:
import pandas as pd
import requests

CEAP

In [8]:
# fazendo o download do conteúdo do arquivo da CEAP
url = f"http://www.camara.leg.br/cotas/Ano-2019.csv.zip"

r = requests.get(url)

with open(f"Ano-2019.csv.zip", "wb") as code:
   code.write(r.content)

In [10]:
# Descompactando o arquivo zip da CEAP
from zipfile import ZipFile

# Descompactando arquivo
arquivo = ZipFile(f"Ano-2019.csv.zip", "r")

lista_itens_arquivo = arquivo.namelist()

arquivo.extract(member=lista_itens_arquivo[0], path=f"reembolso-2019")

arquivo.close()

In [ ]:
# Lendo os dados da CEAP
df_ceap_completo = pd.read_csv("reembolso-2019/Ano-2019.csv", sep=";")

df_ceap_completo.head()

RECEITA FEDERAL

In [13]:
# fazendo o download do conteúdo do arquivo da Receita Federal
import requests

url = "https://serenata-de-amor-data.nyc3.digitaloceanspaces.com/2019-11-19-companies.csv.xz"

r = requests.get(url)

# abrindo um arquivo no seu computador
file = open("2019-11-19-companies.csv.xz", "wb")

# escrever o conteúdo
file.write(r.content)
file.close()

In [ ]:
# Lendo os dados da Receita

DTYPE = {
    'cnpj': str,
}

df_receita_completo = pd.read_csv(
    '2019-11-19-companies.csv.xz', # .xz é um arquivo compactado
    compression='xz',
    dtype=DTYPE,
)
df_receita_completo.head(1)

CRUZANDO OS DADOS DA CEAP COM OS DA RECEITA

In [16]:
df_final = df_ceap_completo.merge(
    df_receita_completo,
    how='left',
    left_on='txtCNPJCPF',
    right_on='cnpj'
)

In [18]:
# Exportando o cruzamento de dados para um arquivo csv
df_final.to_csv("reembolso-2019-completo.csv")

### Some o total da coluna `vlrLiquido` agrupando por `txtCNPJCPF` e `txtFornecedor`

In [30]:
df_final["vlrLiquido"].head()

0    2100.0
1    1200.0
2    1650.0
3    1200.0
4    1200.0
Name: vlrLiquido, dtype: float64

In [32]:
df_final["vlrLiquido"].sum()

150646293.36

In [29]:
df_final["vlrLiquido"].head().sum()

7350.0

In [41]:
df_final.groupby(["txtCNPJCPF","txtFornecedor"])["vlrLiquido"].sum()

txtCNPJCPF          txtFornecedor                                
000.000.000/0000-0  GLOBAL LUXURY SUITES AT KENDALL WEST              4771.51
                    PREFEITURA MUNICIPAL DE FORTALEZA                   78.00
                    sheraton montevieo                                 432.93
000.000.000/0001-0  1919 TAXI CELERITAS                                 32.97
                    ADOBE SYSTEMS SOFTWARE                             348.00
                                                                       ...   
985.911.420/0017-5  Radio Venancio Aires Ltda                         1420.00
985.977.190/0015-6  EMPRESA JORNALISTICA FOLHA DO MATE LTDA - EPP      115.00
986.706.560/0011-6  RESTAURANTE FARINA LTDA                             27.00
991.121.599/72  -   DANYELA LONDERO BERNARDI                           180.00
999.200.001/59  -   Franscielle Maria de Souza                       13500.00
Name: vlrLiquido, Length: 24047, dtype: float64

In [42]:
# Ordenando de forma crescente
df_final.groupby(["txtCNPJCPF","txtFornecedor"])["vlrLiquido"].sum().sort_values(ascending=True)

txtCNPJCPF          txtFornecedor                                                  
301.590.080/0016-6  PAPELARIA E BAZAR SANTA LUZIA                                            2.20
738.300.690/0011-6  J.V. COMÉRCIO DE PETRÓLEO LTDA.                                          2.55
185.722.250/0026-9  Concessionária das Rodovias Centrais do Brasil  S/A                      3.30
046.992.000/0013-5  W P PARK ESTACIONAMENTO LTDA ME                                          3.60
024.154.080/0015-0  CONCESSIONARIA DE RODOVIAS DO OESTE DE SAO PAULO - VIAOESTE S/A          3.80
                                                                                          ...    
026.102.350/0012-0  DMD Gestão Administrativa LTDA - ME                                 805000.00
025.581.570/0016-2  TELEFÔNICA BRASIL S.A. VIVO                                         860813.15
237.210.170/0013-3  T2 COMUNICACAO, VIDEO E PRODUCOES EIRELI - EPP                     1097334.00
020.128.620/0016-0  TAM LINHAS AER

### Ordene os dados da consulta anterior de forma decrescente

In [43]:
df_final.groupby(["txtCNPJCPF","txtFornecedor"])["vlrLiquido"].sum().sort_values(ascending=False)

txtCNPJCPF          txtFornecedor                                                  
073.193.230/0019-1  PANTANAL VEÍCULOS LTDA                                             2181274.52
020.128.620/0016-0  TAM LINHAS AEREAS S/A.                                             1172126.78
237.210.170/0013-3  T2 COMUNICACAO, VIDEO E PRODUCOES EIRELI - EPP                     1097334.00
025.581.570/0016-2  TELEFÔNICA BRASIL S.A. VIVO                                         860813.15
026.102.350/0012-0  DMD Gestão Administrativa LTDA - ME                                 805000.00
                                                                                          ...    
024.154.080/0015-0  CONCESSIONARIA DE RODOVIAS DO OESTE DE SAO PAULO - VIAOESTE S/A          3.80
046.992.000/0013-5  W P PARK ESTACIONAMENTO LTDA ME                                          3.60
185.722.250/0026-9  Concessionária das Rodovias Centrais do Brasil  S/A                      3.30
738.300.690/0011-6  J.V. COMÉRCIO 

### Consulte se houveram gastos reembolsados associados a empresas que na época estavam com status `08`


status == 8 significa `BAIXADA` (*a empresa foi extinta, seja por solicitação dos sócios ou por ação da Receita*)

In [47]:
df_final.head()

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,simples,size,administrative_person_category,name,situation,special_situation,phone,type,state,last_updated
0,AVANTE,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AVANTE,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AVANTE,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVANTE,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AVANTE,NaN,NaN,NaN,2019,NaN,NaN,56,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df_final[df_final["status" == 8]].head()

KeyError: ignored

### Desafio extra

Esse desafio traz algumas ideias de análises. Fique a vontade para tentar encontrar essas respostas :)

- Qual o total gasto nos finais de semana?

- Agrupando por parlamentar e filtrando por finais de semana, qual o parlamentar que mais gastou em um final de semana?

- Tente filtrar os reembolsos envolvendo alimentação, agrupar por fornecedor, e verificar os fornecedores mais frequentados. Poderíamos analisar se existiram gastos com alto desvio padrão considerando os reembolsos de um restaurante em específico. Se você tem o grupo de gastos feitos em um local X, é possível encontrar a média de gasto feito por deputados nesse local X, e de repente encontrar qual teve um gasto muito acima desse valor.

- Podemos tentar encontrar se houve parlamentar que teve alta frequência de reembolsos associados a empresas que já estavam com status `Baixada`.